In [1]:
import sys
sys.path.append('../../../')

In [2]:
import pandas as pd
import numpy as np
import prince
import streamlit as st
import matplotlib.pyplot as plt

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

# Functions

In [3]:
def features_eng(df, version):
    """
    
    """
    if version == 'clustering':
        df = df.drop(columns=['dist2','TransactionID'])
    elif version == 'anomaly':
        df = df.drop(columns=['dist2','customer_id','TransactionID'])
    elif version == 'network':
        df = df.drop(columns=['dist2'])
    df = df.rename(columns={'id_31':'browser'})
    df['P_emaildomain'] = df['P_emaildomain'].mask(df['P_emaildomain']=='gmail','gmail.com')
    df['R_emaildomain'] = df['R_emaildomain'].mask(df['R_emaildomain']=='gmail','gmail.com')
    df['id_30'] = df['id_30'].replace(" ","_",regex=True)
    df['id_30'] = df['id_30'].str.replace(".","_",regex=False)
    df['browser'] = df['browser'].mask(df['browser'].str.contains('SM') | df['browser'].str.contains('ZTE'),'other')
    df['browser'] = df['browser'].astype(str)
    df['browser_enc'] = 'other'
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('ie'),'ie')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('safari'),'safari')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('edge'),'edge')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('firefox'),'firefox')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('android'),'android')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('Android'),'android')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('chrome'),'chrome')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('opera'),'opera')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('google'),'chrome')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('samsung'),'android')
    df['DeviceType'] = df['DeviceType'].mask(df['browser']== 'ie 11.0 for tablet','tablet')
    df = df.drop(columns='browser')
    df['device_info'] = df['DeviceInfo'].replace("-","_",regex=True)
    df['device_info2'] = df['device_info'].replace(" ","_",regex=True)
    df['device_info'] = df['DeviceInfo'].replace("-","_",regex=True)
    df['device_info2'] = df['device_info'].replace(" ","_",regex=True)
    df['device_info3'] = df['device_info2'].str.split("_").str[0]
    df['device_info3'] = df['device_info3'].str.lower()
    df['device_info_v4'] = 'other'
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='windows') | (df['device_info3']=='microsoft') | (df['device_info3']=='trident/7.0')),'windows')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='ios') | (df['device_info3']=='iphone')),'ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='macos','ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='blade','blade')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='lenovo','lenovo')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='redmi','redmi')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='pixel','pixel')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='android','android')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='macos','ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='alcatel','alcatel')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='nokia','nokia')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='asus','asus')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='oneplus','oneplus')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='zte','zte')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='macos','ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='hisense','hisense')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='linux','linux')
    df['device_info_v4'] = df['device_info_v4'].mask((df['device_info3'].str.contains("lg") | (df['device_info3'].str.contains("nexus"))),'lg')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3'].str.contains("huawei")) | (df['device_info3'].str.contains("hi6210sft"))),'huawei')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='motog3') |(df['device_info3']=='moto')),'moto')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='sm') |(df['device_info3']=='samsung')),'samsung')
    df = df.drop(columns=['device_info','device_info2','device_info3','DeviceInfo'])
    return df

# 1.0 Data retrieval

In [4]:
df = pd.read_pickle("app/datalake/train.pkl")
val = pd.read_pickle("app/datalake/val.pkl")

# 2.0 Data preparation

In [5]:
st.write("Starting Features engineering for the training dataset...")
df = features_eng(df,'anomaly')
st.write("Starting Features engineering for the validation dataset...")
val = features_eng(val,'anomaly')

# 3.0 Output

In [6]:
df.to_pickle('app/datalake/train_eng.pkl')
val.to_pickle('app/datalake/val_eng.pkl')